LT 1 PM1

In [4]:
import numpy as np
import pandas as pd
from google.colab import files
import pickle
import matplotlib.pyplot as plt
import random
import io
from numpy import *
import math
data = files.upload()



data1= pd.read_csv(io.StringIO(data['gg.csv'].decode('utf-8'))) #and print the dataset given
data1.isnull().sum()
data1=data1.fillna(value = data1['radius_mean' ].mean())
data1=data1.fillna(value = data1['perimeter_mean' ].mean())
data1=data1.fillna(value = data1['concavity_mean' ].mean())
data1=data1.fillna(value = data1['compactness_se' ].mean())
data1=data1.fillna(value = data1['area_worst' ].mean())
data1=data1.fillna(value = data1['concavity_worst' ].mean())
data1.columns=["ID","Class","f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11","f12","f13","f14","f15","f16","f17","f18","f19","f20","f21","f22","f23","f24","f25","f26","f27","f28","f29","f30"]
data1.pop("ID")
df = data1['Class']
data1.pop("Class")
data1.insert(30, 'Class',df, True)

# i=0
# for line in data1:       #vectorising the dataset
#     if i==0:
#         i=i+1
#         continue
#     line=line.strip('\n')
#     data_line=line.split(',')
#     data_line=[float(i) for i in data_line]
#     dataset.append(data_line[0:31])

dataset = np.array(data1)
dataset = np.hstack((np.ones((dataset.shape[0], 1)), dataset)) #Adding ones to the first column of dataset matrix
#print(dataset)
size_train=int(dataset.shape[0]*0.67)  #Training data size 70% of total
size_test=int(dataset.shape[0]-size_train) #Testing data size 30% of total
#Splitting into train data and test data
x_train,x_test,y_train,y_test=dataset[:size_train,0:31],dataset[size_test:,0:31],dataset[:size_train,31:32],dataset[size_test:,31:32]   


#Function to calculate the accuracy of the model
def calc_accuracy(actual,predicted):
    correct = 0;
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

#Perceptronmodel class
class Perceptron:
    #constructor
    def __init__ (self):
        self.w = None
    
  #Checks wT*x value and its target  
    def model(self, x):
        return 1 if (np.dot(self.w, x) >= 0) else 0
  
  #predictor to predict on the data based on w
    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)
    
    def fit(self, X, Y, epochs = 1, lr = 1):
        #Inititalising w with zeroes
        self.w = np.zeros(X.shape[1])
        #print("Intial w:",self.w)
        accuracy = {}
        max_accuracy = 0
        wt_matrix = []
    #for all epochs
        for i in range(epochs):
            for x, y in zip(X, Y):
                y_pred = self.model(x)
                if y == 1 and y_pred == 0:  #comparing missclassified cases
                    self.w = self.w + lr * x              
                elif y == 0 and y_pred == 1:
                    self.w = self.w - lr * x
                    
            wt_matrix.append(self.w)  
            accuracy[i] = calc_accuracy(self.predict(X), Y)
            if (accuracy[i] > max_accuracy):     #checking for highest accuracy and finding corresponding w
                max_accuracy = accuracy[i]
                chkptw = self.w
                
    #checkpoint to Save the weights
        self.w = chkptw
        print("Max training Accuracy found:", max_accuracy)
        
     #plot the accuracy values over epochs
        #plt.plot(list(accuracy.values()))
        #plt.xlabel("Epoch #")
       # plt.ylabel("Accuracy")
       # plt.ylim([0, 100])
       # plt.show
    #return the weight vectorthat best classifies the points 
        return self.w

perceptron = Perceptron()


wts = perceptron.fit(x_train, y_train, 1000, 0.1)

#making predictions on test data
Y_pred_test = perceptron.predict(x_test)
#checking the testing accuracy of the model
print("\nTesting accuracy score:",calc_accuracy(Y_pred_test, y_test))
print("\nBest w found:",wts)
#random example
Y_NEW = np.dot(wts,x_test[300]);
print("\nPREDICTED VS ACTAL:",Y_NEW,y_test[300]);
def calc_precision_recall(actual, predicted):
    TP, FP, FN, TN = 0, 0, 0, 0
    for i in range(len(actual)):
        if actual[i] == 1 and predicted[i] == 1:
            TP += 1
        elif actual[i] == 0 and predicted[i] == 1:
            FP += 1
        elif actual[i] == 1 and predicted[i] == 0:
            FN += 1
        else:
            TN += 1

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return precision, recall

precision, recall = calc_precision_recall(y_test, Y_pred_test)
print("Precision:", precision)
print("Recall:", recall)


Saving gg.csv to gg (3).csv
Max training Accuracy found: 92.38845144356955

Testing accuracy score: 87.4015748031496

Best w found: [-1.56100000e+02 -1.20305700e+03  1.42234100e+03 -4.50196800e+03
 -3.13780000e+02  9.51669900e+00  8.40042070e+01  1.28941740e+02
  4.96740139e+01  6.99957000e+00  1.97455200e+00 -2.49073400e+01
 -1.21181600e+01  1.96463000e+01  1.31726560e+03  3.22220840e+00
  2.19416528e+01  3.40216170e+01  6.56276640e+00  7.67968990e+00
  1.43838839e+00 -1.26414900e+03  2.14733600e+03 -2.86202300e+03
  1.09078000e+03  2.15220840e+01  2.69464470e+02  3.54140842e+02
  9.23049353e+01  6.25738000e+01  1.91477900e+01]

PREDICTED VS ACTAL: -45052.08376295504 [0.]
Precision: 0.7189542483660131
Recall: 0.9565217391304348
